In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/voting_VA.csv")

print(df.head())
print(df.isnull().sum())

   Unnamed: 0  year     state state_po county_name  county_fips        office  \
0       11161  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
1       11162  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
2       11163  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
3       11164  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
4       11165  2000  VIRGINIA       VA   ALBEMARLE        51003  US PRESIDENT   

        candidate       party  candidatevotes  totalvotes   version   mode  
0         AL GORE    DEMOCRAT            5092       11925  20220315  TOTAL  
1  GEORGE W. BUSH  REPUBLICAN            6352       11925  20220315  TOTAL  
2     RALPH NADER       GREEN             220       11925  20220315  TOTAL  
3           OTHER       OTHER             261       11925  20220315  TOTAL  
4         AL GORE    DEMOCRAT           16255       36846  20220315  TOTAL  
Unnamed: 0        0
year              0
state      

In [2]:
# Remove the 'Unnamed: 0' column
df.drop('Unnamed: 0', axis=1, inplace=True)

# Check unique values for certain columns to ensure consistency
print(df['state'].unique())  # Should only contain "VIRGINIA"
print(df['state_po'].unique())  # Should only contain "VA"
print(df['office'].unique())  # Should only contain "US PRESIDENT"
print(df['mode'].unique())  # Check if any action is needed

['VIRGINIA']
['VA']
['US PRESIDENT']
['TOTAL' 'ABSENTEE' 'ELECTION DAY' 'PROVISIONAL']


In [3]:
df['mode'].head()

0    TOTAL
1    TOTAL
2    TOTAL
3    TOTAL
4    TOTAL
Name: mode, dtype: object

In [4]:
# If we're interested in overall voting results without differentiating between the modes:
# df_total = df[df['mode'] == 'TOTAL']
# df_total.to_csv("data/clean_total_voting_VA.csv", index=False) 

# Otherwise, we can keep all of the modes of voting and use them as features
# in our models
df['mode'] = df['mode'].astype('category')
df['mode'].head()

0    TOTAL
1    TOTAL
2    TOTAL
3    TOTAL
4    TOTAL
Name: mode, dtype: category
Categories (4, object): ['ABSENTEE', 'ELECTION DAY', 'PROVISIONAL', 'TOTAL']

In [5]:
# Provides a dataframe where the modes aggregates all individual records across different voting modes 
# into a single record per candidate per county per election year. 

df_aggregated = df.groupby(['year', 'county_name', 'county_fips', 'candidate', 'party']).agg({
    'candidatevotes': 'sum',
    'totalvotes': 'max'  # Assuming totalvotes is the same across all modes, otherwise sum might be needed
}).reset_index()

df_aggregated.head()

,year,county_name,county_fips,candidate,party,candidatevotes,totalvotes
0,2000,ACCOMACK,51001,AL GORE,DEMOCRAT,5092,11925
1,2000,ACCOMACK,51001,GEORGE W. BUSH,REPUBLICAN,6352,11925
2,2000,ACCOMACK,51001,OTHER,OTHER,261,11925
3,2000,ACCOMACK,51001,RALPH NADER,GREEN,220,11925
4,2000,ALBEMARLE,51003,AL GORE,DEMOCRAT,16255,36846


In [9]:
import pandas as pd
#removing candidates column (unimportant)
df_real = df_aggregated.drop('candidate', axis=1)
#removing third parties (also unimportant)
df_real = df_real[~df_real['party'].isin(['OTHER', 'GREEN'])]
#finding net total votes
df_real['net_total'] = df_real.groupby(['year', 'county_name', 'county_fips', 'totalvotes'])['candidatevotes'].transform(lambda x: x.iloc[1] - x.iloc[0])

# merging columns
df_real = df_real.groupby(['year', 'county_name', 'county_fips', 'totalvotes', 'net_total']).agg({'party': ', '.join}).reset_index()
df_real['winner'] = df_real['net_total'].apply(lambda x: 'REPUBLICAN' if x > 0 else 'DEMOCRAT')
#dropping some more columns
df_real = df_real.drop(['party', 'totalvotes'], axis=1)
df_final = df_real[['year', 'county_name', 'county_fips','net_total', 'winner']]
df_final.head()

,year,county_name,county_fips,net_total,winner
0,2000,ACCOMACK,51001,1260,REPUBLICAN
1,2000,ALBEMARLE,51003,2036,REPUBLICAN
2,2000,ALEXANDRIA,51510,-14590,DEMOCRAT
3,2000,ALLEGHANY,51005,594,REPUBLICAN
4,2000,AMELIA,51007,1193,REPUBLICAN


In [10]:
# county_adjacencies.csv cleaning

df2 = pd.read_csv("data/county_adjacencies.csv")

print(df2.head())
print(df2.isnull().sum())

           County  Population2022   FIPS  District                   N1  \
0        Accomack           33191  51001         2       Northumberland   
1       Albemarle          114534  51003         5  CharlottesvilleCity   
2  AlexandriaCity          155525  51510         8              Fairfax   
3       Alleghany           14835  51005         6                Craig   
4          Amelia           13455  51007         5         PrinceEdward   

           N2         N3             N4           N5        N6      N7  \
0   Lancaster  Middlesex        Mathews  Northampton       NaN     NaN   
1      Nelson    Augusta     Rockingham       Greene    Orange  Louisa   
2   Arlington        NaN            NaN          NaN       NaN     NaN   
3        Bath  Botetourt  CovingtonCity   Rockbridge       NaN     NaN   
4  Cumberland   Powhatan   Chesterfield    Dinwiddie  Nottoway     NaN   

         N8          N9     N10  N11  N12  
0       NaN         NaN     NaN  NaN  NaN  
1  Fluvanna  Buc

In [11]:
# Lots of NaNs for the adjacent neighborhoods

# Fill missing neighbor columns with "None"
neighbor_columns = ['N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10', 'N11', 'N12']
df2[neighbor_columns] = df2[neighbor_columns].fillna("None")

print(df2.isnull().sum())

County            0
Population2022    0
FIPS              0
District          0
N1                0
N2                0
N3                0
N4                0
N5                0
N6                0
N7                0
N8                0
N9                0
N10               0
N11               0
N12               0
dtype: int64


In [12]:
# save this dataset as a csv
# df2.to_csv("data/clean_county_adjacencies.csv", index=False)

In [43]:
#This is another attempt at getting info from csv files into the dataframe. 
csv_files = ["data/county_data/0002_ds176_20105_county_E.csv", "data/county_data/0002_ds191_20125_county_E.csv", "data/county_data/0002_ds191_20125_county_E.csv", "data/county_data/0002_ds191_20125_county_E.csv", "data/county_data/0002_ds239_20185_county_E.csv", "data/county_data/0002_ds249_20205_county_E.csv"]

updated_rows = []
#NOTE: THIS CODE DOES 
for _, row in df_final.iterrows():
    year = row['year']
    county_name = row['county_name']
    fips = row['county_fips']
    csv_file = ""
    if year < 2008:
        csv_file = csv_files[0]
    elif year < 2010:
        csv_file = csv_files[1]
    elif year < 2012:
        csv_file = csv_files[2]
    elif year < 2014:
        csv_file = csv_files[3]
    elif year < 2016:
        csv_file = csv_files[4]
    elif year < 2021:
        csv_file = csv_files[5]
    else:
        print("No csv file for year")
    
    if csv_file:
        df_temp = pd.read_csv(csv_file, low_memory=False)
        df_temp['GEOID'] = df_temp['GEOID'].fillna('')
        df_temp['GEOID_right'] = df_temp['GEOID'].apply(lambda x: x.split("US")[-1])
        
        if not county_row.empty:
            updated_row = pd.concat([row.to_frame().T, county_row.iloc[0].to_frame().T], axis=1)
            updated_rows.append(updated_row)
        else:
            print("Cant find row")
            updated_rows.append(row.to_frame().T)
    else:
        print("Cant open file")
        updated_rows.append(row.to_frame().T)

df_updated = pd.concat(updated_rows, ignore_index=True)

print(df_updated.head())

   year county_name county_fips net_total      winner Unnamed: 0   GISJOIN  \
0  2000    ACCOMACK       51001      1260  REPUBLICAN        NaN       NaN   
1   NaN         NaN         NaN       NaN         NaN       1195  G2300310   
2  2000   ALBEMARLE       51003      2036  REPUBLICAN        NaN       NaN   
3   NaN         NaN         NaN       NaN         NaN       1195  G2300310   
4  2000  ALEXANDRIA       51510    -14590    DEMOCRAT        NaN       NaN   

        YEAR STUSAB REGIONA  ... AMWSE004 AMWSE005 AMWSE006 AMWSE007 AMWSE008  \
0        NaN    NaN     NaN  ...      NaN      NaN      NaN      NaN      NaN   
1  2016-2020     ME     NaN  ...    75671    63890     6507    66695    10893   
2        NaN    NaN     NaN  ...      NaN      NaN      NaN      NaN      NaN   
3  2016-2020     ME     NaN  ...    75671    63890     6507    66695    10893   
4        NaN    NaN     NaN  ...      NaN      NaN      NaN      NaN      NaN   

  AMWSE009 AMWSE010 AMWSE011 AMWSE012 AMWSE0

In [46]:
# printing out all the column names
for column in df_updated.columns:
   print(column)
df_updated.head()

year
county_name
county_fips
net_total
winner
Unnamed: 0
GISJOIN
YEAR
STUSAB
REGIONA
DIVISIONA
STATE
STATEA
COUNTY
COUNTYA
COUSUBA
PLACEA
TRACTA
BLKGRPA
CONCITA
AIANHHA
RES_ONLYA
TRUSTA
AIHHTLI
AITSCEA
ANRCA
CBSAA
CSAA
METDIVA
NECTAA
CNECTAA
NECTADIVA
UAA
CDCURRA
SLDUA
SLDLA
ZCTA5A
SUBMCDA
SDELMA
SDSECA
SDUNIA
PCI
PUMAA
GEOID
NAME
BTTRA
BTBGA
AMPKE001
AMPKE002
AMPKE003
AMPKE004
AMPKE005
AMPKE006
AMPKE007
AMPKE008
AMPKE009
AMPKE010
AMPKE011
AMPKE012
AMPKE013
AMPKE014
AMPKE015
AMPKE016
AMPKE017
AMPKE018
AMPKE019
AMPKE020
AMPKE021
AMPKE022
AMPKE023
AMPKE024
AMPKE025
AMPKE026
AMPKE027
AMPKE028
AMPKE029
AMPKE030
AMPKE031
AMPKE032
AMPKE033
AMPKE034
AMPKE035
AMPKE036
AMPKE037
AMPKE038
AMPKE039
AMPKE040
AMPKE041
AMPKE042
AMPKE043
AMPKE044
AMPKE045
AMPKE046
AMPKE047
AMPKE048
AMPKE049
AMPLE001
AMPLE002
AMPLE003
AMPVE001
AMPWE001
AMPWE002
AMPWE003
AMPWE004
AMPWE005
AMPWE006
AMPWE007
AMPWE008
AMPWE009
AMPWE010
AMZAE001
AMZAE002
AMZAE003
AMZAE004
AMZAE005
AMZAE006
AMZAE007
AMZAE008
AMZAE009
AMZAE01

,year,county_name,county_fips,net_total,winner,Unnamed: 0,GISJOIN,YEAR,STUSAB,REGIONA,...,AMWSE004,AMWSE005,AMWSE006,AMWSE007,AMWSE008,AMWSE009,AMWSE010,AMWSE011,AMWSE012,AMWSE013
0,2000,ACCOMACK,51001,1260,REPUBLICAN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1195,G2300310,2016-2020,ME,NaN,...,75671,63890,6507,66695,10893,3692,300,26,2655,8084
2,2000,ALBEMARLE,51003,2036,REPUBLICAN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1195,G2300310,2016-2020,ME,NaN,...,75671,63890,6507,66695,10893,3692,300,26,2655,8084
4,2000,ALEXANDRIA,51510,-14590,DEMOCRAT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:

columns_to_keep = ['year', 'county_name', 'county_fips','net_total', 'winner',
    'JLZE002', 'JLZE003', 'JLZE004', 'JLZE005', 'JLZE006', 'JLZE007', 'JLZE008', 'JLZE009', 'JLZE010', 'JLZE011', 
    'JLZE012', 'JLZE013', 'JLZE014', 'JLZE015', 'JLZE016', 'JLZE017', 'JLZE018', 'JLZE019', 'JLZE020', 'JLZE021', 
    'JLZE022', 'JLZE023', 'JLZE024', 'JLZE025', 'JLZE026', 'JLZE027', 'JLZE028', 'JLZE029', 'JLZE030', 'JLZE031', 
    'JLZE032', 'JLZE033', 'JLZE034', 'JLZE035', 'JLZE036', 'JLZE037', 'JLZE038', 'JLZE039', 'JLZE040', 'JLZE041', 
    'JLZE042', 'JLZE043', 'JLZE044', 'JLZE045', 'JLZE046', 'JLZE047', 'JLZE048', 'JLZE049', 'JL0E001', 'JL0E002', 
    'JL0E003', 'JMAE001', 'JMBE001', 'JMBE002', 'JMBE003', 'JMBE004', 'JMBE005', 'JMBE006', 'JMBE007', 'JMBE008', 
    'JMBE009', 'JMBE010', 'JMJE001', 'JMJE002', 'JMJE003', 'JMJE004', 'JMJE005', 'JMJE006', 'JMJE007', 'JMJE008', 
    'JMJE009', 'JMJE010', 'JMJE011', 'JMJE012', 'JMJE013', 'JMJE014', 'JMJE015', 'JMJE016', 'JMJE017', 'JMJE018', 
    'JMJE019', 'JMJE020', 'JMJE021', 'JN9E001', 'JN9E002', 'JN9E003', 'JN9E004', 'JN9E005', 'JN9E006', 'JN9E007', 
    'JN9E008', 'JN9E009', 'JN9E010', 'JN9E011', 'JN9E012', 'JN9E013', 'JN9E014', 'JN9E015', 'JN9E016', 'JN9E017', 
    'JN9E018', 'JN9E019', 'JN9E020', 'JN9E021', 'JN9E022', 'JN9E023', 'JN9E024', 'JN9E025', 'JN9E026', 'JN9E027', 
    'JN9E028', 'JN9E029', 'JN9E030', 'JN9E031', 'JN9E032', 'JN9E033', 'JN9E034', 'JN9E035', 'JOCE001', 'JOCE002', 
    'JOCE003', 'JOCE004', 'JOCE005', 'JOCE006', 'JOCE007', 'JOCE008', 'JOHE001', 'JOHE002', 'JOHE003', 'JOHE004', 
    'JOHE005', 'JOHE006', 'JOHE007', 'JOHE008', 'JOHE009', 'JOHE010', 'JOHE011', 'JOHE012', 'JOHE013', 'JOHE014', 
    'JOHE015', 'JOHE016', 'JOHE017', 'JOIE001', 'JQBE001', 'JQLE001', 'JRGE001', 'JRGE002', 'JRGE003', 'JRGE004', 
    'JRGE005', 'JRGE006', 'JRGE007', 'JRGE008', 'JRGE009', 'JRGE010', 'JRGE011', 'JRGE012', 'JRGE013', 'JRGE014', 
    'JRGE015', 'JRGE016', 'JRGE017', 'JRGE018', 'JRGE019', 'JRGE020', 'JRGE021', 'JRGE022', 'JRGE023', 'JRGE024', 
    'JRGE025', 'JRGE026', 'JRGE027', 'JRGE028', 'JRGE029', 'JRGE030', 'JRGE031', 'JRGE032', 'JRGE033', 'JRGE034', 
    'JRGE035', 'JRGE036', 'JRGE037', 'JRGE038', 'JRGE039', 'JRGE040', 'JRGE041', 'JRGE042', 'JRGE043', 'JRGE044', 
    'JRGE045', 'JRGE046', 'JRGE047', 'JRGE048', 'JRGE049', 'JRGE050', 'JRGE051', 'JRGE052', 'JRGE053', 'JRGE054', 
    'JRGE055', 'JRIE001', 'JRJE001', 'JRJE002', 'JRJE003', 'JRKE001', 'JRKE002', 'JRKE003', 'JSAE001', 'JSAE002', 
    'JSAE003', 'JSAE004', 'JSAE005', 'JSAE006', 'JSAE007', 'JSAE008', 'JSAE009', 'JSAE010', 'JSAE011', 'JSDE001', 
    'JSDE002', 'JSDE003', 'JSDE004', 'JSDE005', 'JSDE006', 'JSDE007', 'JSDE008', 'JSDE009', 'JSDE010', 'JS5E001', 
    'JTIE001'
]

df_cleaned = df_updated[columns_to_keep]
df_cleaned.head()

KeyError: "['JLZE002', 'JLZE003', 'JLZE004', 'JLZE005', 'JLZE006', 'JLZE007', 'JLZE008', 'JLZE009', 'JLZE010', 'JLZE011', 'JLZE012', 'JLZE013', 'JLZE014', 'JLZE015', 'JLZE016', 'JLZE017', 'JLZE018', 'JLZE019', 'JLZE020', 'JLZE021', 'JLZE022', 'JLZE023', 'JLZE024', 'JLZE025', 'JLZE026', 'JLZE027', 'JLZE028', 'JLZE029', 'JLZE030', 'JLZE031', 'JLZE032', 'JLZE033', 'JLZE034', 'JLZE035', 'JLZE036', 'JLZE037', 'JLZE038', 'JLZE039', 'JLZE040', 'JLZE041', 'JLZE042', 'JLZE043', 'JLZE044', 'JLZE045', 'JLZE046', 'JLZE047', 'JLZE048', 'JLZE049', 'JL0E001', 'JL0E002', 'JL0E003', 'JMAE001', 'JMBE001', 'JMBE002', 'JMBE003', 'JMBE004', 'JMBE005', 'JMBE006', 'JMBE007', 'JMBE008', 'JMBE009', 'JMBE010', 'JMJE001', 'JMJE002', 'JMJE003', 'JMJE004', 'JMJE005', 'JMJE006', 'JMJE007', 'JMJE008', 'JMJE009', 'JMJE010', 'JMJE011', 'JMJE012', 'JMJE013', 'JMJE014', 'JMJE015', 'JMJE016', 'JMJE017', 'JMJE018', 'JMJE019', 'JMJE020', 'JMJE021', 'JN9E001', 'JN9E002', 'JN9E003', 'JN9E004', 'JN9E005', 'JN9E006', 'JN9E007'